# BUILD A SIMPLE RECOMMENDER

In [2]:
# Import Required Libraries
import pandas as pd   # pandas library for python data handling
import numpy as np
import requests
import json
# Import libraries from scikit learn for feature extraction. 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 

#pretty cell outputs: runs all codes in each cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Getting Data from Yts movie website similar to imbd

Please check the powerpoint file for Web scraping from API of the website 

Link to api:
- https://yts.mx/api
- https://yts.mx/api/v2/list_movies.json?page=1

`Video Tutorial:`

**Intro about google colab**
https://youtu.be/OOpLBlnSLTY

**Scraping list movies data from yts api**
https://youtu.be/QMPsPrK0MDo




## Read json file scraped from the website

In [3]:
df = pd.read_json ("./data/list_movies_new.json")  #read json file  #CHANGE THE FILE PATH AS REQUIRED

In [4]:
df.head() # shows top 5 rows of the dataframe

,id,url,imdb_code,title,title_english,title_long,slug,year,rating,runtime,...,mpa_rating,background_image,background_image_original,small_cover_image,medium_cover_image,large_cover_image,state,torrents,date_uploaded,date_uploaded_unix
0,17273,https://yts.mx/movie/the-big-picture-2010,tt1533818,The Big Picture,The Big Picture,The Big Picture (2010),the-big-picture-2010,2010,6.7,0,...,,https://yts.mx/assets/images/movies/the_big_pi...,https://yts.mx/assets/images/movies/the_big_pi...,https://yts.mx/assets/images/movies/the_big_pi...,https://yts.mx/assets/images/movies/the_big_pi...,https://yts.mx/assets/images/movies/the_big_pi...,ok,[{'url': 'https://yts.mx/torrent/download/000B...,2020-05-03 18:03:11,1.588522e+09
1,17259,https://yts.mx/movie/thappad-2020,tt10964430,Thappad,Thappad,Thappad (2020),thappad-2020,2020,7.1,142,...,,https://yts.mx/assets/images/movies/thappad_20...,https://yts.mx/assets/images/movies/thappad_20...,https://yts.mx/assets/images/movies/thappad_20...,https://yts.mx/assets/images/movies/thappad_20...,https://yts.mx/assets/images/movies/thappad_20...,ok,[{'url': 'https://yts.mx/torrent/download/96B5...,2020-05-03 16:18:57,1.588516e+09
2,17258,https://yts.mx/movie/the-kid-with-a-bike-2011,tt1827512,The Kid with a Bike,The Kid with a Bike,The Kid with a Bike (2011),the-kid-with-a-bike-2011,2011,7.4,87,...,PG-13,https://yts.mx/assets/images/movies/the_kid_wi...,https://yts.mx/assets/images/movies/the_kid_wi...,https://yts.mx/assets/images/movies/the_kid_wi...,https://yts.mx/assets/images/movies/the_kid_wi...,https://yts.mx/assets/images/movies/the_kid_wi...,ok,[{'url': 'https://yts.mx/torrent/download/42FE...,2020-05-03 12:34:18,1.588502e+09
3,17257,https://yts.mx/movie/enter-the-fat-dragon-2020,tt7335600,Enter the Fat Dragon,Enter the Fat Dragon,Enter the Fat Dragon (2020),enter-the-fat-dragon-2020,2020,5.6,96,...,,https://yts.mx/assets/images/movies/enter_the_...,https://yts.mx/assets/images/movies/enter_the_...,https://yts.mx/assets/images/movies/enter_the_...,https://yts.mx/assets/images/movies/enter_the_...,https://yts.mx/assets/images/movies/enter_the_...,ok,[{'url': 'https://yts.mx/torrent/download/EFD1...,2020-05-03 09:37:07,1.588491e+09
4,17256,https://yts.mx/movie/degree-of-murder-1967,tt0061992,Degree of Murder,Degree of Murder,Degree of Murder (1967),degree-of-murder-1967,1967,6.4,87,...,,https://yts.mx/assets/images/movies/degree_of_...,https://yts.mx/assets/images/movies/degree_of_...,https://yts.mx/assets/images/movies/degree_of_...,https://yts.mx/assets/images/movies/degree_of_...,https://yts.mx/assets/images/movies/degree_of_...,ok,[{'url': 'https://yts.mx/torrent/download/0669...,2020-05-03 15:05:46,1.588511e+09


In [6]:
# sorts 'rating' in descending order for whole dataframe to check top rated movies
df.sort_values('rating', ascending=False).head()

,id,url,imdb_code,title,title_english,title_long,slug,year,rating,runtime,genres,summary,description_full,synopsis,yt_trailer_code,language,mpa_rating,background_image,background_image_original,small_cover_image,medium_cover_image,large_cover_image,state,torrents,date_uploaded,date_uploaded_unix
13166,3709,https://yts.mx/movie/the-shawshank-redemption-...,tt0111161,The Shawshank Redemption,The Shawshank Redemption,The Shawshank Redemption (1994),the-shawshank-redemption-1994,1994,9.3,142,"[Action, Crime, Drama]",Chronicles the experiences of a formerly succe...,Chronicles the experiences of a formerly succe...,Chronicles the experiences of a formerly succe...,K_tLp7T6U1c,English,R,https://yts.mx/assets/images/movies/The_Shawsh...,https://yts.mx/assets/images/movies/The_Shawsh...,https://yts.mx/assets/images/movies/The_Shawsh...,https://yts.mx/assets/images/movies/The_Shawsh...,https://yts.mx/assets/images/movies/The_Shawsh...,ok,[{'url': 'https://yts.mx/torrent/download/AC41...,2015-11-01 03:20:23,1.446344e+09
1725,15527,https://yts.mx/movie/top-gear-africa-special-p...,tt2741136,"Top Gear Africa Special, Part 1","Top Gear Africa Special, Part 1","Top Gear Africa Special, Part 1 (2013)",top-gear-africa-special-part-1-2013,2013,9.2,59,"[Adventure, Comedy, Reality-TV, Talk-Show]","In the first of a two-part Top Gear special, J...","In the first of a two-part Top Gear special, J...","In the first of a two-part Top Gear special, J...",,English,,https://yts.mx/assets/images/movies/top_gear_a...,https://yts.mx/assets/images/movies/top_gear_a...,https://yts.mx/assets/images/movies/top_gear_a...,https://yts.mx/assets/images/movies/top_gear_a...,https://yts.mx/assets/images/movies/top_gear_a...,ok,[{'url': 'https://yts.mx/torrent/download/D76F...,2020-02-23 01:25:37,1.582418e+09
13571,3304,https://yts.mx/movie/the-godfather-1972,tt0068646,The Godfather,The Godfather,The Godfather (1972),the-godfather-1972,1972,9.2,175,"[Action, Crime, Drama]","The Godfather ""Don"" Vito Corleone is the head ...","The Godfather ""Don"" Vito Corleone is the head ...","The Godfather ""Don"" Vito Corleone is the head ...",Ak19g0j10-Q,English,,https://yts.mx/assets/images/movies/The_Godfat...,https://yts.mx/assets/images/movies/The_Godfat...,https://yts.mx/assets/images/movies/The_Godfat...,https://yts.mx/assets/images/movies/The_Godfat...,https://yts.mx/assets/images/movies/The_Godfat...,ok,[{'url': 'https://yts.mx/torrent/download/C168...,2015-11-01 02:36:37,1.446342e+09
7465,9717,https://yts.mx/movie/natsamrat-2016,tt5311546,Natsamrat,Natsamrat,Natsamrat (2016),natsamrat-2016,2016,9.1,0,"[Action, Drama, Family]",The film is a tragedy about a veteran theatre ...,The film is a tragedy about a veteran theatre ...,The film is a tragedy about a veteran theatre ...,DCXDyIsPEN8,English,,https://yts.mx/assets/images/movies/natsamrat_...,https://yts.mx/assets/images/movies/natsamrat_...,https://yts.mx/assets/images/movies/natsamrat_...,https://yts.mx/assets/images/movies/natsamrat_...,https://yts.mx/assets/images/movies/natsamrat_...,ok,[{'url': 'https://yts.mx/torrent/download/9C24...,2018-11-12 08:18:51,1.542007e+09
587,16668,https://yts.mx/movie/wheels-2014,tt2170667,Wheels,Wheels,Wheels (2014),wheels-2014,2014,9.0,115,[Drama],"""WHEELS"" is an Award Winning dark humored dram...","""WHEELS"" is an Award Winning dark humored dram...","""WHEELS"" is an Award Winning dark humored dram...",AbRRGP4GJcM,English,R,https://yts.mx/assets/images/movies/wheels_201...,https://yts.mx/assets/images/movies/wheels_201...,https://yts.mx/assets/images/movies/wheels_201...,https://yts.mx/assets/images/movies/wheels_201...,https://yts.mx/assets/images/movies/wheels_201...,ok,[{'url': 'https://yts.mx/torrent/download/708C...,2020-04-13 13:08:54,1.586776e+09


In [7]:
df.shape # <-- check dimension of the dataframe

(16871, 26)

In [8]:
df.columns # <-- check column names of dataframe

Index(['id', 'url', 'imdb_code', 'title', 'title_english', 'title_long',
       'slug', 'year', 'rating', 'runtime', 'genres', 'summary',
       'description_full', 'synopsis', 'yt_trailer_code', 'language',
       'mpa_rating', 'background_image', 'background_image_original',
       'small_cover_image', 'medium_cover_image', 'large_cover_image', 'state',
       'torrents', 'date_uploaded', 'date_uploaded_unix'],
      dtype='object')

# Recommender system
`Recommender system` is a system that seeks to predict or filter preferences according to the user's choices and recommends based on the content. Recommender systems are utilized in a variety of areas including movies, music, news, books, research articles, search queries, social tags, and products in general.

### Types:
- **Popularity based recommendation engine:** eg. trending list in YouTube or Netflix; keeps a track of view counts for each movie/video and then lists movies based on views in descending order(highest view count to lowest view count)

- **Content based recommendation engine:** considers current user's liked movie as input, analyzes the contents of movie (synopsis of movie, genre, cast, director), ranks similar movie based on similarity score and recommends relevant movies to user

- **Collaborative filtering based recommendation engine:** recommends to users based on other similar users

- **Hybrid recommendation system:** combines properties of two or more type of recommendations

## Movie Description Based Recommender

Here, lets try to build a simple content based recommender system using movie descriptions by leveraging scikit-learn library in python. 
Descriptions are qualitative data and requires to be dealt with qualitative method.

In [ ]:
# filling all NaNs with blank string
df['description_full'] = df['description_full'].fillna('')

### Vectorizer: create sparse matrix
Machine learning models requires texts to be in numeric form and we need to represent texts as vectors using `TfidfVectorizer`, a form of count vectorizer which creates sparse matrix based on the frequency of term in the document (rare terms has higher values). “Term Frequency – Inverse Document” Frequency which are the components of the resulting scores assigned to each word. TF-IDF are word frequency scores that try to highlight words that are more interesting, e.g. frequent in a document but not across documents.

In [ ]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['description_full']) #feeding combined strings(movie contents) to TfidfVectorizer() object

In [11]:
tfidf_matrix.shape

(16871, 609244)

### Cosine Similarity

It is used to calculate a numeric quantity that denotes the similarity between two movies. 
Mathematically, it is defined as follows:

cosine(x,y)=x.y⊺/||x||.||y||
 
Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. Therefore, we will use sklearn's linear_kernel instead of cosine_similarities since it is much faster.

In [ ]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [13]:
# similarity value for first movie on the list
cosine_sim[0]

array([1.        , 0.02503509, 0.00534012, ..., 0.00295387, 0.        ,
       0.        ])


**We have a pairwise similarity matrix for all the movies in our dataset. # Lets create a function that returns the most similar movies based on the cosine similarity score.**

In [ ]:
df = df.reset_index()  #reset all index values
titles = df['title_long']  # pass column into titles variable
indices = pd.Series(df.index, index=df['title'])  #get index of the series

In [ ]:
# create a function that returns similar movies, looks into index that we got from above
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

### Getting recommendation of movies

Let us now try and get the top recommendations for a few movies and see how good the recommendations are.

In [16]:
# get title of movie that the user currently likes and give recommendation
get_recommendations('The Entertainers').head(10)

13869             That's My Boy (2012)
11322                Blood Rage (1987)
10430             Teen Wolf Too (1987)
240              The Last Dance (2000)
9801               Rainbow Time (2016)
3088             The Wrong Todd (2018)
14268                        RV (2006)
5267     Infidelity in Suburbia (2017)
5867      Beverly Hills Cop III (1994)
2046     The Barefoot Executive (1971)
Name: title_long, dtype: object

In [17]:
get_recommendations('The Godfather').head(10)

13570                   The Godfather: Part II (1974)
13569                  The Godfather: Part III (1990)
2502                           Family Business (1989)
6779                        The Valachi Papers (1972)
11823                            The Seven-Ups (1973)
9245                                 Roommates (1995)
15478    Halloween: The Curse of Michael Myers (1995)
9668                                 Linewatch (2008)
9662                        The Last Godfather (2010)
972                      After All These Years (2013)
Name: title_long, dtype: object

**Result:** The sytem is able to identify `The Godfather` as Godfather movie and recommends other godfather movies as its top recommendations. 
We can improve the system by including other important features such as cast, crew, director and genre, which determine the rating and the popularity of a movie.